# Universities Visiting Scheduler

This Notebook is an assignement for the Coursea Data Science Course.

## Description

Assume that you represent an education organization which aims to improve the education system by collecting and analyzing information on different universities.

This mission involves paying visits to universities in different cities.

In order to make the best use of the limited time and budget, it might be helpful have a scheduler that allows you to visit universities in the same areas each time. 

In this case, I'll use the universities in San Francisco as an example.

## Data

In order to perform such task,
* I'll be using Geocoders to obtain coordinates of the city. 
* I'll be using the Foursquare API to obtain name and coordinates information from the venue information.
* A k-means clustering will be applied to segment the universities based on their coordinates information.

## Methodology

#### 1. Import libraries those will be used in this Notebook.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 

import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm 
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from sklearn.cluster import KMeans

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00   1.10 MB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.64 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   2.94 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  22.81 MB/s
vincent-0.4.4- 100% |###################

#### 2. Setting destination info

List of available categories can be found on FOURSQUARE website https://developer.foursquare.com/docs/resources/categories

In [26]:
target_category = ['University']  #Setting category
quanity = 20 #Max number of universities aim to visit
period = 4 #Number of days for the trip
city = 'San Francisco, CA, United States'

#### 3. Get coordinates of the city

In [3]:
geolocator = Nominatim()
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of ', city, ' are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The coordinates of  San Francisco, CA, United States  are 37.7647993, -122.4629897.


#### 4. Fetch a list of Universities in Target City with FOURSQUARE API

In [5]:
#parepare the crendentials
CLIENT_ID = 'VD0PE51PROYUONWZKUEAC5QQQQBG42SNKEANTD54ZGL3EUGU' # your Foursquare ID
CLIENT_SECRET = 'NAME4KBXAEEKTKQUKSTFQ55CIUXCSVWPZ55X2V1IGGLMZREK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VD0PE51PROYUONWZKUEAC5QQQQBG42SNKEANTD54ZGL3EUGU
CLIENT_SECRET:NAME4KBXAEEKTKQUKSTFQ55CIUXCSVWPZ55X2V1IGGLMZREK


In [11]:
#prepare category request url
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
category_result = requests.get(url).json()

In [39]:
#fetching category id
category_list = []
category_id = []
for c1 in category_result['response']['categories']:
    if c1['name'] in target_category:
        category_list.append([c1['name'],c1['id']])
        category_id.append(c1['id'])
    for c2 in c1['categories']:
        if c2['name'] in target_category:
                category_list.append([c2['name'],c2['id']])
                category_id.append(c2['id'])
        if (len(c2['categories']) != 0):
            for c3 in c2['categories']:
                if c3['name'] in target_category:
                    category_list.append([c3['name'],c3['id']])
                    category_id.append(c3['id'])
category_list

[['University', '4bf58dd8d48988d1ae941735']]

In [42]:
# fethcing list of targets
LIMIT= 50
radius = 10000  #10km

results = []
for index, cid in enumerate(category_id):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            cid,
            radius, 
            LIMIT)
    results.append(requests.get(url).json())
results

[{'meta': {'code': 200, 'requestId': '5be663759fb6b71ed152ab85'},
  'response': {'confident': True,
   'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
        'suffix': '.png'},
       'id': '4bf58dd8d48988d1ae941735',
       'name': 'University',
       'pluralName': 'Universities',
       'primary': True,
       'shortName': 'University'}],
     'hasPerk': False,
     'id': '50324dabe4b0d68f3901092a',
     'location': {'cc': 'US',
      'city': 'San Francisco',
      'country': 'United States',
      'distance': 448,
      'formattedAddress': ['San Francisco, CA 94134', 'United States'],
      'labeledLatLngs': [{'label': 'display',
        'lat': 37.76435268260494,
        'lng': -122.45792803460328}],
      'lat': 37.76435268260494,
      'lng': -122.45792803460328,
      'postalCode': '94134',
      'state': 'CA'},
     'name': 'UCSF Parking Garage - P5',
     'referralId': 'v-1541825397'},
    {'categories': [{'icon': {'

#### 5. Convert List to Dataframe

In [19]:
university_list=[]
for res in results:
    if (len(res['response'])  != 0):
        for v in res['response']['venues']:
            if (len(v['categories']) != 0):
                university_list.append((
                    v['name'],         
                    v['location']['lat'], 
                    v['location']['lng'],
                    v['categories'][0]['name'],
                    v['id']))
                
university_list = pd.DataFrame(university_list)
university_list.columns = ['Name', 'Latitude', 'Longtitude', 'Category','Id']

In [25]:
university_list.head()

,Name,Latitude,Longtitude,Category,Id
0,UCSF Parking Garage - P5,37.764353,-122.457928,University,50324dabe4b0d68f3901092a
1,UCSF - School Of Nursing,37.762711,-122.458632,University,4bccb069cc8cd13aea64c1cf
2,"University of California, San Francisco (UCSF)",37.763313,-122.458398,University,4c54894d06901b8d83d0f64b
3,UCSF Department of Pathology,37.762474,-122.458452,University,585438d22ec3647ab973e61a
4,Eli and Edythe Broad Center of Regeneration Me...,37.762030,-122.457877,University,4df6a00baeb7da11e1a69b8a


#### 6. Clustring based on Latiude and Longitude

In [23]:
#clustering_venues=top_travel_venues.loc[:,['Name','Category', 'Latitude','Longtitude']]
clustering_universities=university_list.loc[:,['Latitude','Longtitude']]
clustering_universities.head()

,Latitude,Longtitude
0,37.764353,-122.457928
1,37.762711,-122.458632
2,37.763313,-122.458398
3,37.762474,-122.458452
4,37.762030,-122.457877


In [28]:
# set number of clusters
kclusters = period

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, n_init = 12).fit(clustering_universities)


# check cluster labels generated for each row in the dataframe
kmeans.labels_

universities_grouped = university_list.loc[:,['Name', 'Category', 'Latitude','Longtitude']]

# add clustering labels
universities_grouped['Cluster Labels'] = kmeans.labels_
universities_grouped.head()

,Name,Category,Latitude,Longtitude,Cluster Labels
0,UCSF Parking Garage - P5,University,37.764353,-122.457928,0
1,UCSF - School Of Nursing,University,37.762711,-122.458632,0
2,"University of California, San Francisco (UCSF)",University,37.763313,-122.458398,0
3,UCSF Department of Pathology,University,37.762474,-122.458452,0
4,Eli and Edythe Broad Center of Regeneration Me...,University,37.762030,-122.457877,0


#### 7. Display results on the map

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(universities_grouped['Latitude'], universities_grouped['Longtitude'], universities_grouped['Name'], universities_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 8. List out a schedule

**Day 1**

In [31]:
universities_grouped.loc[universities_grouped['Cluster Labels'] == 0, ['Name', 'Category']]

,Name,Category
0,UCSF Parking Garage - P5,University
1,UCSF - School Of Nursing,University
2,"University of California, San Francisco (UCSF)",University
3,UCSF Department of Pathology,University
4,Eli and Edythe Broad Center of Regeneration Me...,University
11,I see you lil' punk ass,University
15,Academy Of Art University - Palace Of Fine Arts,University
17,USF Presidio Campus,University
25,USF - University of San Francisco,University
27,UCSF Parking Garage P8 Purple,University


**Day 2**

In [32]:
universities_grouped.loc[universities_grouped['Cluster Labels'] == 1, ['Name', 'Category']]

,Name,Category
6,UCSF Mission Bay,Medical School
8,Northwestern University San Francisco,University
9,Academy of Art University - Powell,Art Gallery
12,UCSF Mission Bay - Sandler Neurosciences Bldg,University
16,"University of the Pacific, Dugoni School of De...",Medical School
18,USF - Downtown Campus,University
19,Academy of Art University - Brannan,College Arts Building
20,Wharton | San Francisco,University
21,RocketSpace Innovation Campus,University
22,INSEEC,University


**Day 3**

In [33]:
universities_grouped.loc[universities_grouped['Cluster Labels'] == 2, ['Name', 'Category']]

,Name,Category
5,San Francisco State University (SFSU),University
37,SFSU Anthropology Student Lounge,University
47,"City College of San Francisco, Ocean Campus",Community College


**Day 4**

In [34]:
universities_grouped.loc[universities_grouped['Cluster Labels'] == 3, ['Name', 'Category']]

,Name,Category
7,California Institute of Integral Studies,College Academic Building
10,Santa Clara University,University
13,Make School,University
14,Stoop,University
31,Alliance Health Project,Medical Center
35,Zac Citte,University
40,CTI Coaching Certification Call,University


## Discussion 

It's probably because Foursquare's data is crowsourced, some of the reulsts are very interesting, like result 11, which is obviously not a university, but classified as university.

Further data preprocessing can be applied to improve accuracy of the results.

## Conclusion

This might be a quite short Notebook, but it should serve its purpuse very well. The results are clustered with the coordinates, so targets with close distance are grouped together, which should save the user the time to go back and forth. The list of visiting targets and a map with their location should provide a clear idea to the audience.